# Table of Contents
 <p><div class="lev1"><a href="#VARIABLE-DESCRIPTIONS:"><span class="toc-item-num">1&nbsp;&nbsp;</span>VARIABLE DESCRIPTIONS:</a></div>

## Setup

In [1]:
!pip install -q numpy scikit-learn ipython matplotlib pandas sh

# VARIABLE DESCRIPTIONS:

* survival        Survival            (0 = No; 1 = Yes)
* pclass          Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
* name            Name
* sex             Sex
* age             Age
* sibsp           Number of Siblings/Spouses Aboard
* parch           Number of Parents/Children Aboard
* ticket          Ticket Number
* fare            Passenger Fare
* cabin           Cabin
* embarked        Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
 1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
 If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored.  The following are the definitions used
for sibsp and parch.

Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent:   Mother or Father of Passenger Aboard Titanic
Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws.  Some children travelled
only with a nanny, therefore parch=0 for them.  As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.

In [2]:
# This Python 3 environment comes with many helpful analytics libraries 
# installed
# It is defined by the kaggle/python docker image: 
# https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from IPython.display import display
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, accuracy_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    FunctionTransformer, OneHotEncoder, StandardScaler,
    MinMaxScaler, MaxAbsScaler
)
from sklearn.svm import SVC

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) 
# will list the files in the input directory

import sh
print(sh.ls('./input'))

genderclassmodel.csv  gendermodel.csv  test.csv  train.csv  train.csv.ori



In [3]:
Y_test =  pd.read_csv('./input/genderclassmodel.csv')
test =  pd.read_csv('./input/test.csv')
train =  pd.read_csv('./input/train.csv')

print('test set has %s rows and %s columns.' % test.shape)
print('test set has %s rows and %s columns.' % Y_test.shape)
print('train set has %s rows and %s columns.' % train.shape)

test set has 418 rows and 11 columns.
test set has 418 rows and 2 columns.
train set has 891 rows and 12 columns.


In [4]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
def prepare_df(df, p_unique=0.2):
    df = df.copy()
    
    size = len(df)
    
    df = pd.get_dummies(df, drop_first=True)
    cols = list(df.columns)
    
    for c in cols:
        if df[c].dtype == object:
            if df[c].nunique()/size > p_unique:
                df.drop(c, axis=1, inplace=True)
                continue
            
    return df

In [6]:
pca = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='mean'),
    StandardScaler(),
    PCA(random_state=42)
)

pca.fit_transform(prepare_df(train.drop('Survived', axis=1)), train.Survived);

In [7]:
pca.get_params()

{'memory': None,
 'steps': [('simpleimputer', SimpleImputer()),
  ('standardscaler', StandardScaler()),
  ('pca', PCA(random_state=42))],
 'verbose': False,
 'simpleimputer': SimpleImputer(),
 'standardscaler': StandardScaler(),
 'pca': PCA(random_state=42),
 'simpleimputer__add_indicator': False,
 'simpleimputer__copy': True,
 'simpleimputer__fill_value': None,
 'simpleimputer__keep_empty_features': False,
 'simpleimputer__missing_values': nan,
 'simpleimputer__strategy': 'mean',
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'pca__copy': True,
 'pca__iterated_power': 'auto',
 'pca__n_components': None,
 'pca__n_oversamples': 10,
 'pca__power_iteration_normalizer': 'auto',
 'pca__random_state': 42,
 'pca__svd_solver': 'auto',
 'pca__tol': 0.0,
 'pca__whiten': False}

In [8]:
train_ = train.copy()

train_.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
# train_.dropna(inplace=True)

X_train = prepare_df(train_.drop('Survived', axis=1))
y_train = train_[['Survived']]

In [9]:
test_ = pd.concat([Y_test, test.iloc[:, 1:]], axis=1)

test_.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
# test_.dropna(inplace=True)

X_test = prepare_df(test_.drop('Survived', axis=1))
y_test = test_[['Survived']]

In [10]:
test_.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')

In [11]:
print('X_test set has %s rows and %s columns.' % X_test.shape)
print('y_test set has %s rows and %s columns.' % y_test.shape)
print('X_train set has %s rows and %s columns.' % X_train.shape)
print('Y_train set has %s rows and %s columns.' % y_train.shape)

X_test set has 418 rows and 8 columns.
y_test set has 418 rows and 1 columns.
X_train set has 891 rows and 8 columns.
Y_train set has 891 rows and 1 columns.


In [12]:
# , MaxAbsScaler
pipeline_rfc = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='mean'),
    MaxAbsScaler(),
    RandomForestClassifier(random_state=42)
)

pipeline_svc = make_pipeline(
    SimpleImputer(missing_values=np.nan, strategy='mean'),
    MaxAbsScaler(),
    SVC()
)

In [13]:
for title, model in {
    'RandomForestClassifier': pipeline_rfc, 
    'SVC': pipeline_svc
}.items():
    print('\n%s' % model)
    model.fit(X_train, y_train)

    predicted = model.predict(X_test)
    print('f1 score')
    print(f1_score(y_test, predicted))
    print('precision score')
    print(precision_score(y_test, predicted))
    print('accuracy score')
    print(accuracy_score(y_test, predicted))
    f_name = 'output/%s.csv' % title
    
    # np.savetxt(f_name, predicted)
    # print('%s saved!' % f_name)


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('maxabsscaler', MaxAbsScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])


/home/xmn/miniforge3/envs/datateller-web/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


f1 score
0.75
precision score
0.7161290322580646
accuracy score
0.8229665071770335

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('maxabsscaler', MaxAbsScaler()), ('svc', SVC())])
f1 score
0.873015873015873
precision score
0.990990990990991
accuracy score
0.9234449760765551


/home/xmn/miniforge3/envs/datateller-web/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
